# Number of same moves within CGP evolution

In [19]:
import tengp
import numpy as np
from sklearn.metrics import mean_squared_error

In [42]:
from functools import partial

@tengp.utils.handle_invalid_decorator
def simple_es_moves(X, y, cost_function, params,
              target_fitness=0,
              population_size=5,
              evaluations=5000,
              random_state=None,
              mutation='point',
              mutation_probability=0.25,
              verbose=False):

    if mutation not in tengp.mutations.MUTATIONS:
        raise tengp.utils.UnknownMutationException("Provided type of mutation is not implemented.")

    move = tengp.mutations.MUTATIONS[mutation]
    if mutation == 'probabilistic':
        move = partial(move, probability=mutation_probability)

    if random_state:
        random.seed(random_state)

    # initial generation
    ib = tengp.individual.IndividualBuilder(params)

    population = [ib.create() for _ in range(population_size)]

    n_evals = 0

    generation = 0

    for individual in population:
        output = individual.transform(X)
        individual.fitness = cost_function(y, output)
        n_evals += 1


    all_moves = []
    while n_evals < evaluations:
        generation += 1

        parent = min(population, key=lambda x: x.fitness)

        if parent.fitness <= target_fitness:
            return population

        population = []
        for _ in range(population_size - 1):
            new_move = move(parent)
            all_moves.append(new_move)
            population.append(parent.apply(new_move))
            
        population = [parent.apply(move(parent)) for _ in range(population_size - 1)]

        population += [parent]

        for individual in population:
            output = individual.transform(X)
            individual.fitness = cost_function(y, output)
            n_evals += 1

        if verbose and generation % 100 == 0:
            print(f'Gen: {generation}, population: {sorted([x.fitness for x in population])}')

    population.sort(key=lambda x: x.fitness)
    return population, all_moves


In [11]:
from gpbenchmarks import get_data

In [7]:
X, y = get_data('nguyenf10', 20, -1, 1)

In [14]:
X = np.c_[np.ones(len(X)), X]

Define the function set and CGP parameters

In [51]:
funset = tengp.FunctionSet()
funset.add(np.add, 2)
funset.add(np.subtract, 2)
funset.add(np.multiply, 2)
funset.add(np.divide, 2)
funset.add(np.sin, 1)
funset.add(np.cos, 1)

params = tengp.Parameters(3, 1, 1, 10, funset)

In [45]:
def same_moves(moves):
    hits = 0
    for move in moves:
        for m in moves:
            if id(m) == id(move):
                continue
            if move == m:
                hits += 1
    return hits

Same moves for **point mutation**:

In [52]:
population, moves = simple_es_moves(X, y, mean_squared_error, params, mutation='point')

print(len(moves), same_moves(moves))

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:113: RuntimeWarning: divide by zero encountered in true_divide
  current_node.value = current_node.fun(*values)
/home/jarino/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


3996 87594


Same moves for single mutation

In [53]:
population, moves = simple_es_moves(X, y, mean_squared_error, params, mutation='single')

print(len(moves), same_moves(moves))

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:113: RuntimeWarning: divide by zero encountered in true_divide
  current_node.value = current_node.fun(*values)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:113: RuntimeWarning: invalid value encountered in true_divide
  current_node.value = current_node.fun(*values)
/home/jarino/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:113: RuntimeWarning: invalid value encountered in sin
  current_node.value = current_node.fun(*values)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:113: RuntimeWarning: invalid value encountered in cos
  current_node.value = current_node.fun(*values)


3996 27178


Same moves for **probabilistic mutation**:

In [54]:
population, moves = simple_es_moves(X, y, mean_squared_error, params, mutation='probabilistic')

print(len(moves), same_moves(moves))

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:113: RuntimeWarning: divide by zero encountered in true_divide
  current_node.value = current_node.fun(*values)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:113: RuntimeWarning: invalid value encountered in true_divide
  current_node.value = current_node.fun(*values)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:113: RuntimeWarning: invalid value encountered in cos
  current_node.value = current_node.fun(*values)
/home/jarino/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


3996 0
